In [2]:
import torch
import datasets
import pandas as pd
from trl import SFTTrainer
from peft import LoraConfig
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
device = 'cpu'
if(torch.cuda.is_available()):
    device = 'cuda'

print(device)

In [ ]:
Train_Path = "DFAR dataset/DFAR_trainset.csv"
Test_Path = "DFAR dataset/DFAR_testset.csv"

train_set = pd.read_csv(Train_Path, encoding='utf-8')
test_set = pd.read_csv(Test_Path, encoding='utf-8')

In [4]:
train_set['Label'] = train_set['Label'].replace({0: "ethical", 1: "unethical"})
test_set['Label'] = test_set['Label'].replace({0: "ethical", 1: "unethical"})

In [7]:
train_set = datasets.Dataset.from_dict(train_set)
test_set = datasets.Dataset.from_dict(test_set)

In [11]:
def finetune_labelreason_prompts(sample):
    bos_token = "<s>"
    
    system_message = "Evaluate whether the statement is ethical or unethical with reason."
    statement = sample["Statement"]
    response = "The statement is "+ sample["Label"]+"."+ sample["Reason"]
    eos_token = "</s>"

    full_prompt = ""
    full_prompt += bos_token
    full_prompt += "\n\n### Instruction:"
    full_prompt += "\n " + system_message
    full_prompt += "\n\n### Input:"
    full_prompt += "\n " + statement
    full_prompt += "\n\n### Response:"
    full_prompt += "\n " + response
    full_prompt += eos_token

    return full_prompt


In [ ]:
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=False
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [41]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
# Training Params
args = TrainingArguments(
    output_dir = "./checkpoints",
    num_train_epochs = 10,
    per_device_train_batch_size = 4, 
    gradient_accumulation_steps = 1,
    optim="paged_adamw_32bit",
    save_steps = 25,
    logging_steps = 25,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    fp16 = False,
    bf16 = False,
    max_grad_norm = 0.3,
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    lr_scheduler_type = "constant",
)

In [47]:
max_seq_length = 1000

trainer = SFTTrainer(
  model = model,
  peft_config = peft_config,
  max_seq_length = max_seq_length,
  tokenizer = tokenizer,
  packing = True,
  formatting_func = finetune_labelreason_prompts,
  args = args,
  train_dataset = train_set, 
  eval_dataset = test_set    
)

In [ ]:
trainer.train()